In [8]:
import sounddevice as sd
from scipy.io.wavfile import write
import numpy as np
# import tk
from pynput import keyboard

# Recording some sounds

Starting with sample recording

In [24]:
fs = 44100  # Sample rate
seconds = 2  # Duration of recording

myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
sd.wait()  # Wait until recording is finished

rec2 = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
sd.wait()  # Wait until recording is finished
#write('samples/output.wav', fs, myrecording)  # Save as WAV file 

In [44]:
# Example from array
myrecording[1900]

array([0.00643921, 0.0072937 ], dtype=float32)

In [11]:
type(myrecording)
myrecording.shape

(88200, 2)

In [34]:
# Record 10 pices and add them in memory
i = 0
record_flag = True
fist_run = True
sample_duration = 1
full_recording = sd.rec(int(1), samplerate=fs, channels=2)
old_recording = full_recording.copy()
print(full_recording)
while (record_flag):
    new_recording = sd.rec(int(sample_duration * fs), samplerate=fs, channels=2)
    sd.wait()  # Wait until recording is finished
    full_recording = np.concatenate((full_recording,new_recording))
    i += 1
    if i > 10:
        record_flag = False
    

[[0.00018311 0.00375366]]


In [35]:
write('samples/10s.wav', fs, full_recording)  # Save as WAV file 

# Trying with keyboard
Simple example of keypress detection

In [64]:
from pynput import keyboard
def on_press(key):
    try:
        print('Alphanumeric key pressed: {0} '.format(
            key.char))
    except AttributeError:
        print(type(key))
        print('special key pressed: {0}'.format(
            key))

def on_release(key):
    print('Key released: {0}'.format(
        key))
    if key == keyboard.Key.esc:
        # Stop listener
        return False
    
# Collect events until released
# with keyboard.Listener(
#         on_press=on_press,
#         on_release=on_release) as listener:
#     listener.join()


# ...or, in a non-blocking fashion:
listener = keyboard.Listener(
    on_press=on_press,
    on_release=on_release)
listener.start()    
    
print("EXECUTE")


EXECUTE
Alphanumeric key pressed: e 
Key released: 'e'
Alphanumeric key pressed: a 
Key released: 'a'
<enum 'Key'>
special key pressed: Key.space
Key released: Key.space
<enum 'Key'>
special key pressed: Key.esc
Key released: Key.esc


In [42]:
# To stop listener
listener.stop()

special key pressed: Key.enter
Key released: Key.enter


# Now lets do some keypress recording!

In [ ]:
import time

time.sleep(0.4)
### Settings ###
fs = 44100  # Sample rate
seconds = 2  # Duration of recording
recordings_count = 3
recordings_after_the_press = 1
sample_duration = 1



def on_press(key):
    try:
        print('Alphanumeric key pressed: {0} '.format(
            key.char))
        # if (key.char == "s"):
        #     print("STOPPING")
    except AttributeError:
        print('special key pressed: {0}'.format(
            key))

def on_release(key):
    global stop_flag
    print('Key released: {0}'.format(
        key))
    stop_flag = True
    if key == keyboard.Key.esc:
        # Stop listener
        return False
    
stop_flag = False
record_flag = True

stop_initiated = False
fist_run = True

listener = keyboard.Listener(
    on_press=on_press,
    on_release=on_release)
listener.start()

# Allocate memory for moving recordings list

recordings = [sd.rec(int(1), samplerate=fs, channels=2)]
i = 0

while (record_flag):
    new_recording = sd.rec(int(sample_duration * fs), samplerate=fs, channels=2)
    if len(recordings) > recordings_count:
        recordings.pop(0)
    sd.wait()  # Wait until recording is finished
    recordings.append(new_recording)
    # print(f"record flag: {record_flag}")
    if (stop_flag):
        i += 1
        if i > recordings_after_the_press:
            record_flag = False
    

full_recording = np.concatenate(recordings)



In [ ]:
write('samples/test_key_press.wav', fs, full_recording)  # Save as WAV file 

# Test of the result

In [65]:
from get_key_sample import *

In [ ]:
listener = keyboard.Listener(
    on_press=on_press,
    on_release=on_release)
listener.start()
logging.info(f'Listener started')


rec = record_next_press()

listener.stop()

# Working with database

My small playing ground!

In [88]:
import pandas as pd
df = pd.read_csv("dataset.csv")

In [89]:
df

,id,path,key_pressed,num_of_samples,time_created,comment
0,22,123,k,1,10000,NaN
1,23,123,k,1,10001,NaN


In [92]:
int(df.iloc[-1:]["id"])

23

In [105]:
new_database_entery = {
        "id":45,
        "path":"125",
        "key_pressed":"f",
        "num_of_samples":1000,
        "time_created":123,
        "comment":""
    }

In [106]:
df.append(new_database_entery,ignore_index=True)

/tmp/ipykernel_12263/1036307934.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(new_database_entery,ignore_index=True)


,id,path,key_pressed,num_of_samples,time_created,comment
0,22,123,k,1,10000,NaN
1,23,123,k,1,10001,NaN
2,45,125,f,1000,123,


In [103]:
schema = ["id","path","key_pressed","num_of_samples","time_created","comment"]
database = pd.DataFrame(columns = schema)

In [122]:
pd.DataFrame(new_database_entery,index=[0])
# df = pd.DataFrame({'A': 1, 'B': 2}, index=[0])

,id,path,key_pressed,num_of_samples,time_created,comment
0,45,125,f,1000,123,


In [132]:
database = pd.concat([database,pd.DataFrame(new_database_entery,index=[0])])

In [131]:
database

,id,path,key_pressed,num_of_samples,time_created,comment
0,45,125,f,1000,123,
0,45,125,f,1000,123,
0,45,125,f,1000,123,
